<a href="https://colab.research.google.com/github/manote101/Building-Apps-with-NIM/blob/main/Lab-03_RAG_Concept/32-RAG_LangChain-FAISS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction to LangChain

In [1]:
# For Colab user
import os
from google.colab import userdata
os.environ['NVIDIA_API_KEY'] = userdata.get('NVIDIA_API_KEY')

In [2]:
# download sample data
!git clone https://github.com/manote101/Building-Apps-with-NIM.git

Cloning into 'Building-Apps-with-NIM'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 46 (delta 11), reused 36 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (46/46), 2.51 MiB | 14.14 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [3]:
!pip install -r Building-Apps-with-NIM/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.8/450.8 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.1.0
    Uninstalling langchain-core-1.1.0:
      Successfully uninstalled langchain-core-1.1.0
  Attempting uninstall: langchain
    Found existing installation: langchain 1.1.0
    Uninstalling langchain-1.1.0:
      Successfully uninstalled langchain-1.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langgra

In [4]:
#pip install -q --upgrade langchain langchain-community langchain-nvidia_ai_endpoints faiss-cpu requests==2.32.4

from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_core.runnables import RunnableMap, RunnablePassthrough

# 1. Load and split documents
loader = TextLoader("Building-Apps-with-NIM/data/doc1.txt")  # replace with your file
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(docs)

# 2. Create embeddings and vectorstore
embeddings = NVIDIAEmbeddings(model="nvidia/llama-3.2-nv-embedqa-1b-v2")
vectorstore = FAISS.from_documents(splits, embeddings)

# 3. Create retriever
retriever = vectorstore.as_retriever()

# 4. Create LLM and prompt
llm = ChatNVIDIA(model="meta/llama-3.2-3b-instruct", temperature=0)

prompt = ChatPromptTemplate.from_template("""
You are a helpful AI assistant. Use the following context to answer the user's question.

Context:
{context}

Question:
{question}

Answer:
""")

# 5. Create a retrieval-augmented chain
retrieval_chain = RunnableMap({
    # "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "context": lambda x: retriever.invoke(x["question"]),  # Use invoke()
    "question": RunnablePassthrough(),  # Pass the question unchanged
}) | prompt | llm

# 6. Call invoke() to run it
result = retrieval_chain.invoke({"question": "บริษัทใดมีการนำ Nemo microservices มาใข้แล้วบ้าง"})

# 7. Print result
print(result.content)


ตามข้อมูลที่ให้มา บริษัทต่อไปนี้มีการนำ NeMo microservices มาใช้แล้ว:

1. Cloudera
2. Datadog
3. Dataiku
4. DataRobot
5. DataStax
6. SuperAnnotate AI
7. Weights & Biases
8. AT&T (ใช้ NeMo Customizer และ Evaluator เพื่อเพิ่มความแม่นยำของ AI Agent โดย fine-tune โมเดล Mistral 7B สำหรับบริการส่วนบุคคล)
